In [1]:
#Install Libraries
!pip install openai --q
!pip install nasim[dqn] --q
!pip install pyyaml --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 53.8 MB/s eta 0:00:00


In [2]:
###TINY EXAMPLE

#Make imports
import os
from openai import OpenAI
import yaml
import subprocess
import shutil
import nasim
import time
#Load instructions and relative example
filename1 = '/content/instruction.txt'
filename2 = '/content/example.txt'
# Read the content of the first text file
with open(filename1, 'r') as file:
    txt = file.read()

# Read the content of the second text file
with open(filename2, 'r') as file:
    txt2 = file.read()
#set api
client = OpenAI(
    api_key='sk-n*******************************',
)
text = f"Given these instructions: {txt} I get this network: {txt2}"

generator = "Given this example: " + txt + ' create a new one with different setting. Do not provide explanations. do not prompt anything before hosts number'



/usr/local/lib/python3.10/dist-packages/nasim/envs/render.py:17: UserWarning: Unable to import Matplotlib with TkAgg backend due to following exception: "<class 'ImportError'> Cannot load backend 'TkAgg' which requires the 'tk' interactive framework, as 'headless' is currently running". NASIM can still run but GUI functionallity may not work as expected.
  warnings.warn(


FileNotFoundError: [Errno 2] No such file or directory: '/content/instruction.txt'

In [ ]:
#Example-Based Configuration

store_output=[]
db=[]
status=[]
safe=[]
user_input = f"now generate a new network configuration for this example: {txt}. Do not prompt 'Here is a new network configuration based on the provided requirements:'".

        if safe ==[]:
          text_content = "Given these instructions: " + text + 'Learn this relationship and get ready to produce more of those.'
        else:
          #this is current reinforcment system
          text_content = "Given these instructions: " + text + 'Learn this relationship and get ready to produce more of those.' + 'These configurations are successful:'+ str(safe)

        stream = client.chat.completions.create(
          model="gpt-4-turbo",
          messages=[
              {"role": "system", "content": txt},
              {"role": "user", "content": user_input}
          ],
          stream=True)
        output_text = ""

        for chunk in stream:
          if chunk.choices[0].delta.content is not None:
              output_text += chunk.choices[0].delta.content
        #print(output_text)

        with open('1.yaml', 'w') as file:
          file.write(output_text)
        tiny_yaml_path = '/usr/local/lib/python3.10/dist-packages/nasim/scenarios/benchmark/tiny.yaml'
        new_yaml_path='/content/1.yaml'
        shutil.copy(new_yaml_path, tiny_yaml_path)

        result = subprocess.run(['python', '-m', 'nasim.demo', 'tiny'], capture_output=True, text=True)
        if result.stdout == '':
          print ('ERROR !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
          db.append(output_text)
          status.append('Error')
        else:
          db.append(output_text)
          safe.append(output_text)
          status.append('Success')
          print ('SUCCESS !!!!!!!!!!!!!!!!!!!!!!!!!!!!')
db.to_csv('1.csv')

In [ ]:
#Expanded Example-Based Configuration
user_input = f"now generate a new network configuration for this example: {txt}. Do not prompt 'Here is a new network configuration based on the provided requirements:'".

store_output=[]
db=[]
status=[]
safe=[]
        if safe ==[]:
          text_content = "Given these instructions: " + text + 'Learn this relationship and get ready to produce more of those.' +read_csv + reflect (Exploration: Prompt the LLM to generate multiple candidate responses to the original query, without conditioning on previous outputs.
Reflection: For each candidate response, prompt the LLM to provide a self-critique, identifying potential flaws or limitations in its reasoning.
Revision: Based on the response-critique pairs, prompt the LLM to generate an improved final output that resolves the identified issues.)
        else:
          #this is current reinforcment system
          text_content = "Given these instructions: " + text + 'Learn this relationship and get ready to produce more of those.' + 'These configurations are successful:'+ +read_csv + reflect + str(safe)

        stream = client.chat.completions.create(
          model="gpt-4-turbo",
          messages=[
              {"role": "system", "content": txt},
              {"role": "user", "content": user_input}
          ],
          stream=True)
        output_text = ""

        for chunk in stream:
          if chunk.choices[0].delta.content is not None:
              output_text += chunk.choices[0].delta.content
        #print(output_text)

        with open('1.yaml', 'w') as file:
          file.write(output_text)
        tiny_yaml_path = '/usr/local/lib/python3.10/dist-packages/nasim/scenarios/benchmark/tiny.yaml'
        new_yaml_path='/content/1.yaml'
        shutil.copy(new_yaml_path, tiny_yaml_path)

        result = subprocess.run(['python', '-m', 'nasim.demo', 'tiny'], capture_output=True, text=True)
        if result.stdout == '':
          print ('ERROR !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
          db.append(output_text)
          status.append('Error')
        else:
          db.append(output_text)
          safe.append(output_text)
          status.append('Success')
          print ('SUCCESS !!!!!!!!!!!!!!!!!!!!!!!!!!!!')
db.to_csv('2.csv')


In [ ]:
#Explicit Geometric Network Topology (EGNT) Approach

store_output=[]
db=[]
status=[]
safe=[]
topological_enanchment=  'The sum of subnets must be equal to the number of hosts. The network topology must be represented by a matrix of dimensions (1 + len(subnets)) x (1 + len(subnets)).'
user_input = f"now generate a new network configuration for this example: {txt}. {topological_enanchment} Do not prompt 'Here is a new network configuration based on the provided requirements:'. Refer to the documentation link: https://networkattacksimulator.readthedocs.io/en/latest/tutorials/scenarios.html#scenarios-tute"



        if safe ==[]:
          text_content = "Given these instructions: " + text + 'Learn this relationship and get ready to produce more of those.'
        else:
          #this is current reinforcment system
          text_content = "Given these instructions: " + text + 'Learn this relationship and get ready to produce more of those.' + 'These configurations are successful:' + str(safe)

        stream = client.chat.completions.create(
          model="gpt-4-turbo",
          messages=[
              {"role": "system", "content": txt},
              {"role": "user", "content": user_input}
          ],
          stream=True)
        output_text = ""

        for chunk in stream:
          if chunk.choices[0].delta.content is not None:
              output_text += chunk.choices[0].delta.content
        #print(output_text)

        with open('1.yaml', 'w') as file:
          file.write(output_text)
        tiny_yaml_path = '/usr/local/lib/python3.10/dist-packages/nasim/scenarios/benchmark/tiny.yaml'
        new_yaml_path='/content/1.yaml'
        shutil.copy(new_yaml_path, tiny_yaml_path)

        result = subprocess.run(['python', '-m', 'nasim.demo', 'tiny'], capture_output=True, text=True)
        if result.stdout == '':
          print ('ERROR !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
          db.append(output_text)
          status.append('Error')
        else:
          db.append(output_text)
          safe.append(output_text)
          status.append('Success')
          print ('SUCCESS !!!!!!!!!!!!!!!!!!!!!!!!!!!!')
db.to_csv('3.csv')







In [ ]:
#EGNT Approach + Expanded Example-Based Configuration (load filtered positive)


store_output=[]
db=[]
status=[]
safe=[]
topological_enanchment=  'The sum of subnets must be equal to the number of hosts. The network topology must be represented by a matrix of dimensions (1 + len(subnets)) x (1 + len(subnets)).'
user_input = f"now generate a new network configuration for this example: {txt}. {topological_enanchment} Do not prompt 'Here is a new network configuration based on the provided requirements:'. Refer to the documentation link: https://networkattacksimulator.readthedocs.io/en/latest/tutorials/scenarios.html#scenarios-tute"



        if safe ==[]:
          text_content = "Given these instructions: " + text + 'Learn this relationship and get ready to produce more of those.' + pd.read_csv(filtered)
        else:
          #this is current reinforcment system
          text_content = "Given these instructions: " + text + 'Learn this relationship and get ready to produce more of those.' + 'These configurations are successful:' + + pd.read_csv(filtered) + str(safe)

        stream = client.chat.completions.create(
          model="gpt-4-turbo",
          messages=[
              {"role": "system", "content": txt},
              {"role": "user", "content": user_input}
          ],
          stream=True)
        output_text = ""

        for chunk in stream:
          if chunk.choices[0].delta.content is not None:
              output_text += chunk.choices[0].delta.content
        #print(output_text)

        with open('1.yaml', 'w') as file:
          file.write(output_text)
        tiny_yaml_path = '/usr/local/lib/python3.10/dist-packages/nasim/scenarios/benchmark/tiny.yaml'
        new_yaml_path='/content/1.yaml'
        shutil.copy(new_yaml_path, tiny_yaml_path)

        result = subprocess.run(['python', '-m', 'nasim.demo', 'tiny'], capture_output=True, text=True)
        if result.stdout == '':
          print ('ERROR !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
          db.append(output_text)
          status.append('Error')
        else:
          db.append(output_text)
          safe.append(output_text)
          status.append('Success')
          print ('SUCCESS !!!!!!!!!!!!!!!!!!!!!!!!!!!!')
db.to_csv('4.csv')
